In [ ]:
# import relevant libraries
!pip install numerapi
!pip install ondemand
!pip install py_vollib
!pip install selenium
import os
import re
import sys
import glob
import json
import time
import math
import shutil
import config
import random
import pickle
import ondemand
import numerapi
import datetime
import py_vollib
import numpy as np
import scipy as sp
import pandas as pd
import requests as rq
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from os import system   
from math import floor
from scipy import stats
from copy import deepcopy
from datetime import datetime
from sklearn import linear_model
from sklearn import neural_network
from sklearn import ensemble
from bs4 import BeautifulSoup as bs
from sklearn.decomposition import PCA
from statsmodels.formula.api import ols
from sklearn import metrics as sklMetrics
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

plt.style.use('seaborn')
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

## Load Data

In [ ]:
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

In [ ]:
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
tournament_data.head()

In [ ]:
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

In [ ]:
training_features = training_data[feature_cols]
live_features = tournament_data[feature_cols]

## Sneetches Model (Multi-Layer Perceptron)

In [ ]:
predictions = neural_network.MLPRegressor(max_iter=500).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchModID)

## Same As Model (Gradient-Boosted tree)

In [ ]:
predictions = ensemble.GradientBoostingRegressor(n_estimators=500).fit(training_features, training_data.target).predict(live_features)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SameAsModID)

## Sneetches Without Model (PCA/Gradient-Boosted Tree)

In [ ]:
pca_model = PCA(n_components=30).fit(training_features, training_data.target)
pca_train_x = pca_model.transform(training_features)
pca_test_x = pca_model.transform(live_features)
predictions = ensemble.GradientBoostingRegressor(n_estimators=500).fit(pca_train_x, training_data.target).predict(pca_test_x)

In [ ]:
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions

In [ ]:
napi = numerapi.NumerAPI(public_id=config.PubKey, secret_key=config.SecKey)
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=config.SneetchWOModID)